In [ ]:
import os
import yaml
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
# Path ke folder dataset (ganti sesuai lokasi dataset di PC Anda)
dataset_path = "E:\kuliah\SEM 9\PCD\tgs\gambar"
yaml_path = os.path.join(dataset_path, "data.yaml")  # File YAML
train_dir = os.path.join(dataset_path, "train")      # Folder training
valid_dir = os.path.join(dataset_path, "valid")      # Folder validasi


In [ ]:
# Fungsi untuk membaca file YAML
def read_yaml(yaml_path):
    with open(yaml_path, 'r') as file:
        data = yaml.safe_load(file)
    return data

In [ ]:
# Fungsi untuk memuat gambar dan label
def load_images_and_labels(data_dir):
    images = []
    labels = []
    class_names = os.listdir(data_dir)
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(data_dir, class_name)
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            if file_name.endswith(('.png', '.jpg', '.jpeg')):
                img = cv2.imread(file_path)
                img = cv2.resize(img, (100, 100))  # Resize gambar ke ukuran 100x100
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Konversi BGR ke RGB
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels), class_names

In [ ]:
# Muat data training
print("Memuat data training...")
X_train, y_train, class_names_train = load_images_and_labels(train_dir)
print(f"Data training: {X_train.shape[0]} gambar")


In [ ]:
# Muat data validasi
print("Memuat data validasi...")
X_valid, y_valid, class_names_valid = load_images_and_labels(valid_dir)
print(f"Data validasi: {X_valid.shape[0]} gambar")

In [ ]:
# Periksa kesesuaian kelas
def check_class_consistency(classes_train, classes_valid):
    if set(classes_train) != set(classes_valid):
        raise ValueError("Kelas di data training dan validasi tidak cocok!")

check_class_consistency(class_names_train, class_names_valid)

In [ ]:
# Normalisasi data
X_train = X_train / 255.0
X_valid = X_valid / 255.0


In [ ]:
# Flatten gambar menjadi vektor fitur
X_train_flatten = X_train.reshape(X_train.shape[0], -1)
X_valid_flatten = X_valid.reshape(X_valid.shape[0], -1)


In [ ]:
# Bangun model KNN
print("Membangun model KNN...")
k = 3  # Jumlah tetangga terdekat
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train_flatten, y_train)


In [ ]:
# Evaluasi model
print("Evaluasi model...")
y_pred = knn.predict(X_valid_flatten)
accuracy = accuracy_score(y_valid, y_pred)
print(f"Akurasi: {accuracy:.2f}")
print("\nLaporan Klasifikasi:")
print(classification_report(y_valid, y_pred, target_names=class_names_train))

In [ ]:
# Visualisasi beberapa hasil prediksi
def visualize_predictions(X, y_true, y_pred, class_names, num_images=5):
    indices = np.random.choice(range(len(X)), num_images, replace=False)
    for i, idx in enumerate(indices):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(X[idx])
        plt.title(f"True: {class_names[y_true[idx]]}\nPred: {class_names[y_pred[idx]]}")
        plt.axis('off')
    plt.show()

visualize_predictions(X_valid, y_valid, y_pred, class_names_train)